In [10]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [11]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [15]:
# Ler os dados de um arquivo txt
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
 print('Arquivo "{name}" de tamanho {length} bytes foi adicionado.'.format( name=fn, length=len(uploaded[fn])))
from pyspark import SparkContext, SparkConf

Saving lipsum.txt to lipsum.txt
Arquivo "lipsum.txt" de tamanho 29926 bytes foi adicionado.


In [18]:
# Cria a aplicação e instacia o SparkContext
conf = SparkConf().setAppName("ContaPalavras")
sc = SparkContext.getOrCreate()

# Cria o RDD com o conteúdo do lipsum.txt
conteudoRDD = sc.textFile("lipsum.txt")

# Elimina as linha em branco
remove_linha_vazia = conteudoRDD.filter(lambda x: len(x) > 0)

In [46]:
# Separa as palavras usando o espaço em branco entre elas
palavras = remove_linha_vazia.flatMap(lambda x:x.split(' '))
# Map-Reduce da contagem das palavras
contapalavras = palavras.map(lambda x:(x[0],1)) \
.reduceByKey(lambda x, y: x + y) \
.map(lambda x: (x[1], x[0])).sortByKey(False)
# Imprime o resultado, depois de ter contado a quantidade de ocorrências de cada letra
for palavra in contapalavras.collect():
  print(palavra)

(433, 'e')
(354, 'a')
(307, 'v')
(286, 's')
(275, 'l')
(273, 'n')
(237, 'm')
(234, 'i')
(233, 'p')
(198, 't')
(177, 'd')
(175, 'f')
(164, 'c')
(138, 'u')
(81, 'q')
(80, 'o')
(73, 'N')
(73, 'P')
(69, 'D')
(67, 'M')
(64, 'S')
(54, 'r')
(48, 'b')
(45, 'I')
(43, 'C')
(37, 'h')
(35, 'A')
(34, 'V')
(28, 'j')
(24, 'Q')
(20, 'E')
(18, 'g')
(15, 'F')
(15, 'U')
(4, 'L')
(2, 'O')


In [42]:
print(palavras.cache)

<bound method RDD.cache of PythonRDD[178] at RDD at PythonRDD.scala:53>
